# Itinerary Basics

Basic report on stage itinerary - stage name, distance etc.

In [1]:
import notebookimport

In [2]:
import os
import sqlite3
import pandas as pd

In [3]:
if __name__=='__main__':
    #dbname='wrc18.db'
    dbname='mexico18.db'
    conn = sqlite3.connect(dbname)
    rally='Mexico'

In [4]:
#!/Users/ajh59/anaconda3/bin/pip install tracery
import tracery
from tracery.modifiers import base_english
from inflectenglish import inflect_english
from pytracery_logic import pytracery_logic

def pandas_row_mapper(row, rules, root,  modifiers=base_english):
    ''' Function to parse single row of dataframe '''
    row=row.to_dict()
    rules=rules.copy()

    for k in row:
        rules[k] = str(row[k])
        grammar = tracery.Grammar(rules)
        if modifiers is not None:
            if isinstance(modifiers,list):
                for modifier in modifiers:
                    grammar.add_modifiers(modifier)
            else:
                grammar.add_modifiers(modifiers)

    return grammar.flatten(root)

def traceryparse(rules, root, modifiers=base_english):
    grammar = tracery.Grammar(rules)
    if modifiers is not None:
        if isinstance(modifiers,list):
            for modifier in modifiers:
                grammar.add_modifiers(modifier)
        else:
            grammar.add_modifiers(modifiers)
    
    return grammar.flatten(root)
    

## Rally Summary

In [5]:
def dbGetRallySummary(rally):
    q='''
    SELECT o.*, t.totalDistance FROM (SELECT ce.`country.name`, ce.startDate, ce.finishDate, ce.name, ce.`country.iso3`, ce.surfaces,
    COUNT(*) numOfStages, SUM(itc.distance) AS compDistanceKm
    FROM itinerary_controls itc
    JOIN championship_events ce ON itc.eventId=ce.eventId
    WHERE ce.`country.name`="{rally}" AND type='StageStart') AS o
    JOIN (SELECT ce.`country.name`, SUM(itc.distance) totalDistance FROM itinerary_controls itc
    JOIN championship_events ce ON itc.eventId=ce.eventId
    WHERE ce.`country.name`="{rally}") AS t ON o.`country.name` = t.`country.name`
    '''.format(rally=rally)
    
    rallydetails = pd.read_sql(q,conn)
    
    return rallydetails

In [6]:
rs = dbGetRallySummary(rally)
rs

,country.name,startDate,finishDate,name,country.iso3,surfaces,numOfStages,compDistanceKm,totalDistance
0,Mexico,2018-03-08,2018-03-11,Rally Guanajuato México,MEX,Gravel,22,344.49,1055.88


In [7]:
rs.dtypes

country.name       object
startDate          object
finishDate         object
name               object
country.iso3       object
surfaces           object
numOfStages         int64
compDistanceKm    float64
totalDistance     float64
dtype: object

In [8]:
rules = {'origin': "#name# (#startDate# to #finishDate#) #stages#. #dist#. #surface#.",
         'stages': "runs over #numOfStages# competitive special stages",
         'dist': "The distance covered on the special stages is #compDistanceKm.round#km, with an overall rally distance of #totalDistance.round#km",
         'surface':"The special stage surface type is predominantly #surfaces#",
        }

In [9]:
rs['report'] = rs.apply(lambda row: pandas_row_mapper(row, rules, "#origin#",[base_english,inflect_english, pytracery_logic]), axis=1)

In [10]:
rs[rs['country.name']==rally]['report'].iloc[0]

'Rally Guanajuato México (2018-03-08 to 2018-03-11) runs over 22 competitive special stages. The distance covered on the special stages is 344.49km, with an overall rally distance of 1055.88km. The special stage surface type is predominantly Gravel.'

In [11]:
basepath = 'report'
if not os.path.exists(basepath):
    os.makedirs(basepath)

In [12]:
README='''# Rally Report - {}

*This report is unofficial and is not associated in any way with the Fédération Internationale de l’Automobile (FIA) or WRC Promoter GmbH.*


{}
'''.format(rally, rs[rs['country.name']==rally]['report'].iloc[0])

with open('{}/README.md'.format(basepath), 'a') as out_file:
    out_file.write(README)

In [13]:
import inflect
p=inflect.engine()

In [14]:
rs['startDate']=pd.to_datetime(rs['startDate'])
rs['startDate'].loc[0].strftime("%A %d,  %B %Y")

'Thursday 08,  March 2018'

In [15]:
p.number_to_words(p.ordinal((int(rs['startDate'].loc[0].strftime("%d")))))

'eighth'

## Itinerary Items

In [16]:
def dbGetTimeControls(rally):

    q='''
    SELECT il.name AS date, itc.*, ce.timeZoneOffset,
         isc.itinerarySectionId, isc.name AS section, isc.`order`
    FROM itinerary_controls itc
    JOIN championship_events ce ON itc.eventId=ce.eventId
    JOIN itinerary_sections isc ON itc.`itinerarySections.itinerarySectionId`=isc.itinerarySectionId
    JOIN itinerary_legs il ON isc.itineraryLegId=il.itineraryLegId
    WHERE ce.`country.name`="{rally}" AND firstCarDueDateTimeLocal NOT NULL ORDER BY firstCarDueDateTimeLocal 
    '''.format(rally=rally)
    time_controls = pd.read_sql(q,conn)
    time_controls['firstCarDueDateTimeLocal']=pd.to_datetime(time_controls['firstCarDueDateTimeLocal'])
    return time_controls

In [17]:
q='''
    SELECT *
    FROM itinerary_controls itc
    JOIN championship_events ce ON itc.eventId=ce.eventId
    JOIN itinerary_sections isc ON itc.`itinerarySections.itinerarySectionId`=isc.itinerarySectionId
    JOIN itinerary_legs il ON isc.itineraryLegId=il.itineraryLegId
    WHERE ce.`country.name`="{rally}" AND firstCarDueDateTimeLocal NOT NULL ORDER BY firstCarDueDateTimeLocal 
    '''.format(rally=rally)
xx =pd.read_sql(q,conn)
xx.columns

Index(['code', 'controlId', 'controlPenalties', 'distance', 'eventId',
       'firstCarDueDateTime', 'firstCarDueDateTimeLocal', 'location',
       'stageId', 'status', 'targetDuration', 'targetDurationMs',
       'timingPrecision', 'type', 'itineraryLegId',
       'itinerarySections.itinerarySectionId', 'categories',
       'clerkOfTheCourse', 'country.countryId', 'country.iso2', 'country.iso3',
       'country.name', 'countryId', 'eventId', 'finishDate', 'location',
       'mode', 'name', 'organiserUrl', 'slug', 'startDate', 'stewards',
       'surfaces', 'templateFilename', 'timeZoneId', 'timeZoneName',
       'timeZoneOffset', 'trackingEventId', 'itineraryLegId',
       'itinerarySectionId', 'name', 'order', 'itineraryId', 'itineraryLegId',
       'legDate', 'name', 'order', 'startListId', 'status'],
      dtype='object')

In [18]:
import datetime
t=datetime.time(abs(int(360/60)), 360 % 60)
dt = datetime.datetime.combine(datetime.date.today(), t)
dt.isoformat()

'2018-03-26T06:00:00'

In [19]:
t = datetime.datetime.combine(datetime.date.today(),datetime.time(0))
(t  + datetime.timedelta( minutes=-359)).isoformat()

'2018-03-25T18:01:00'

In [20]:
xx[:2][['timeZoneId','timeZoneOffset']]

,timeZoneId,timeZoneOffset
0,Central Standard Time (Mexico),-360
1,Central Standard Time (Mexico),-360


In [21]:
if __name__=='__main__':
    time_controls = dbGetTimeControls(rally)
    display(time_controls.head())

,date,code,controlId,controlPenalties,distance,eventId,firstCarDueDateTime,firstCarDueDateTimeLocal,location,stageId,...,targetDuration,targetDurationMs,timingPrecision,type,itineraryLegId,itinerarySections.itinerarySectionId,timeZoneOffset,itinerarySectionId,section,order
0,Thursday 8 March,TC0,1453,All,NaN,28,2018-03-09T00:15:00,2018-03-09 00:15:00,Rally Campus Leon,-1,...,00:00:00,0,Minute,TimeControl,60,134,-360,134,Section 1,1
1,Thursday 8 March,TC0A,1452,All,34.57,28,2018-03-09T01:05:00,2018-03-09 01:05:00,Silao Square IN,-1,...,00:50:00,3000000,Minute,TimeControl,60,134,-360,134,Section 1,1
2,Thursday 8 March,TC0B,1451,All,0.19,28,2018-03-09T01:20:00,2018-03-09 01:20:00,Silao Square OUT,-1,...,00:15:00,900000,Minute,TimeControl,60,134,-360,134,Section 1,1
3,Thursday 8 March,TC1,1509,All,23.00,28,2018-03-09T02:00:00,2018-03-09 02:00:00,Teatro Juarez,378,...,00:40:00,2400000,Minute,TimeControl,60,134,-360,134,Section 1,1
4,Thursday 8 March,SS1,1515,All,2.53,28,2018-03-09T02:08:00,2018-03-09 02:08:00,Monster Street Stage GTO,378,...,00:08:00,480000,Minute,StageStart,60,134,-360,134,Section 1,1


In [22]:
#Check datetime type
time_controls['firstCarDueDateTime'] = pd.to_datetime(time_controls['firstCarDueDateTime'])

In [23]:
import datetime

def newtime(row):
    t=datetime.timedelta( minutes=row['timeZoneOffset'])
    return row['firstCarDueDateTime']+ t

time_controls['mylocaltime'] = time_controls.apply(lambda row: newtime(row),axis=1)

In [24]:
rules = {'origin': "#mylocaltime.pdtime(%H:%M:%S)# #code# #location# #distance.isNotNull(post=km).brackets# \[#targetDuration#\]",
        }

In [25]:
dategroups = time_controls.groupby('date',sort=False)
for key in dategroups.groups.keys():
    print('---\n\n{}:\n'.format(key))
    grouped2=dategroups.get_group(key).groupby('section',sort=False)
    for key2 in grouped2.groups.keys():
        g2 = grouped2.get_group(key2)
        l=len(g2[g2['code'].str.startswith('SS')])
        print('{} - {} special {}\n'.format(key2,p.number_to_words(l),p.plural_noun('stage',l)))
        for r in grouped2.get_group(key2).apply(lambda row: pandas_row_mapper(row, rules, "#origin#",[base_english,inflect_english]), axis=1):
            print('\t{}'.format(r))
        print('\n')

---

Thursday 8 March:

Section 1 - one special stage

	18:15:00 TC0 Rally Campus Leon  [00:00:00]
	19:05:00 TC0A Silao Square IN (34.57km) [00:50:00]
	19:20:00 TC0B Silao Square OUT (0.19km) [00:15:00]
	20:00:00 TC1 Teatro Juarez (23.0km) [00:40:00]
	20:08:00 SS1 Monster Street Stage GTO (2.53km) [00:08:00]
	21:38:00 TC1A Overnight Regroup IN (55.27km) [01:30:00]


---

Friday 9 March:

Section 2 - four special stages

	09:30:00 TC1B Overnight Regroup OUT-Service A IN  [00:00:00]
	09:45:00 TC1C Service  A OUT  [00:15:00]
	10:30:00 TC2 Las Coloradas (30.08km) [00:45:00]
	10:33:00 SS2 Duarte - Derramadero 1 (26.05km) [00:03:00]
	11:18:00 TC3 Cuestecita (11.69km) [00:45:00]
	11:21:00 SS3 El Chocolate 1 (31.44km) [00:03:00]
	12:11:00 TC4 El Zorrillo (7.33km) [00:50:00]
	12:14:00 SS4 Ortega 1 (17.23km) [00:03:00]
	13:59:00 TC5 Rally Campus Leon (65.09km) [01:45:00]
	14:02:00 SS5 Street Stage Leon 1 (1.11km) [00:03:00]
	14:22:00 TC5A Technical Zone IN-Regroup IN (1.05km) [00:20:00]


Sectio

In [26]:
sectionREADME_base = '''### {section} Report
'''

SUMMARY ='\n'

rules['stages'] = "#code# - #location# #distance.isNotNull(post=km).brackets#"

sections = time_controls.groupby('section',sort=False)
for key in sections.groups.keys():
    
    sectionfn = '{bp}/{key}_report.md'.format(bp=basepath,key=key)
    
    with open(sectionfn, 'w') as out_file:
        out_file.write('')
        
    sectionControls = sections.get_group(key)
    sstages = sectionControls[sectionControls['code'].str.startswith('SS')]
    l=len(sstages)
    title = '# {}, {}\n\nThis section comprises {} special {}'.format(key,sectionControls['date'].iloc[0], p.number_to_words(l),p.plural_noun('stage',l))
    sectionREADME = '''{title}'''.format( title=title)
    
    sstage=[]
    for r in sstages.apply(lambda row: pandas_row_mapper(row, rules, "#stages#",[base_english,inflect_english]), axis=1):
        sstage.append(r)
    sectionREADME = '{} ({})'.format(sectionREADME,', '.join(sstage))

    
    sectionREADME = '''{s}\n\nThe full scheduled itinerary for the section was as follows:\n'''.format(s=sectionREADME,)
    
    controls = []
    for r in sectionControls.apply(lambda row: pandas_row_mapper(row, rules, "#origin#",[base_english,inflect_english]), axis=1):
        controls.append(r)
    sectionREADME = '{}\n\t- {}\n'.format(sectionREADME, '\n\t- '.join(controls))
      
    print(sectionREADME,'\n------\n')
    with open(sectionfn, 'a') as out_file:
        out_file.write(sectionREADME)
        
    #Add section
    SUMMARY = '{summary}\n* [{key}]({key}_report.md)\n'.format(summary=SUMMARY, key=key)
    
    sstageDict = sstages.set_index('code').to_dict(orient='index')
    #Add special stages
    for s in sstageDict: #[Section 2](Section 2_report.md)
        SUMMARY = '{summary}  - [{s} - {n}]({s}_report.md)\n'.format(summary=SUMMARY,s=s, n=sstageDict[s]['location'])
    
with open('{}/SUMMARY.md'.format(basepath), 'a') as out_file:
    out_file.write(SUMMARY)


# Section 1, Thursday 8 March

This section comprises one special stage (SS1 - Monster Street Stage GTO (2.53km))

The full scheduled itinerary for the section was as follows:

	- 18:15:00 TC0 Rally Campus Leon  [00:00:00]
	- 19:05:00 TC0A Silao Square IN (34.57km) [00:50:00]
	- 19:20:00 TC0B Silao Square OUT (0.19km) [00:15:00]
	- 20:00:00 TC1 Teatro Juarez (23.0km) [00:40:00]
	- 20:08:00 SS1 Monster Street Stage GTO (2.53km) [00:08:00]
	- 21:38:00 TC1A Overnight Regroup IN (55.27km) [01:30:00]
 
------

# Section 2, Friday 9 March

This section comprises four special stages (SS2 - Duarte - Derramadero 1 (26.05km), SS3 - El Chocolate 1 (31.44km), SS4 - Ortega 1 (17.23km), SS5 - Street Stage Leon 1 (1.11km))

The full scheduled itinerary for the section was as follows:

	- 09:30:00 TC1B Overnight Regroup OUT-Service A IN  [00:00:00]
	- 09:45:00 TC1C Service  A OUT  [00:15:00]
	- 10:30:00 TC2 Las Coloradas (30.08km) [00:45:00]
	- 10:33:00 SS2 Duarte - Derramadero 1 (26.05km) [00:03:00]


In [27]:
sstages.set_index('code').to_dict(orient='index')[]

SyntaxError: invalid syntax (<ipython-input-27-fab559f70820>, line 1)

In [ ]:
print(SUMMARY)

In [ ]:
!head -n100 "report/Section 3_report.md"